# Sentiment Analysis using Naive Bayes

In this assignment, we will attempt to label tweets with sentiments (positive, neutral and negative) using Naive Bayes classifier. Naive Bayes is a very basic approach to this problem, but gives surprisingly good accuracy sometimes.

**Fill in the Blanks**

## Importing required libraries

In [1]:
import pandas as pd
import re
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

## Reading dataset

In [2]:
from google.colab import files
uploaded = files.upload()

Saving tweets.csv to tweets.csv


In [35]:
import io
data=pd.read_csv(io.BytesIO(uploaded['tweets.csv']))

data.drop(data.columns[0],axis=1,inplace=True)
print(data.shape)
#data.dropna(subset = ["tweets"], inplace=True)
print(data.shape)

(1380, 2)
(1380, 2)


## Text processing for the tweets

In [36]:
import nltk 
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [48]:
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 

stopwords = set(stopwords.words('english') + list(punctuation) + ['AT_USER','URL'])
    
def processTweet(tweet):
    # tweet is the text we will pass for preprocessing 
    # convert passed tweet to lower case 

    tweet=str(tweet).lower()
    tweet
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
    tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
    
    # use work_tokenize imported above to tokenize the tweet
    tweet =word_tokenize(tweet)
   
    return [word for word in tweet if word not in stopwords]

## Process all tweets

In [ ]:
stopwords

In [49]:
processed=[]

for tweet in data['tweets']:
    
    # process all tweets using processTweet function above - store in variable 'cleaned' 
    cleaned=processTweet(tweet)
    
    processed.append(' '.join(cleaned))

In [50]:

data['processed'] = processed


## Create pipeline and define parameters for GridSearch

In [51]:
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])

tuned_parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': [1, 1e-1, 1e-2]
}

## Split data into test and train

In [52]:
# split data into train and test with split as 0.2 
X = data.processed
y = data.labels

x_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


## Perform classification (using GridSearch)

In [53]:



clf = GridSearchCV(text_clf,
                    tuned_parameters, 
                    cv = 10
                    
                    ) 

clf.fit(x_train, y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        pre

## Classification report 

In [61]:
# print classification report after predicting on test set with best model obtained in GridSearch
 
best_model = clf.best_estimator_
print(classification_report(y_test, best_model.predict(X_test)))

              precision    recall  f1-score   support

           0       0.86      0.97      0.91       194
           1       0.88      0.68      0.77        68
           2       0.83      0.36      0.50        14

    accuracy                           0.87       276
   macro avg       0.86      0.67      0.73       276
weighted avg       0.87      0.87      0.86       276



## Important:

In [62]:
counts = data.labels.value_counts()
print(counts)

0    947
1    352
2     81
Name: labels, dtype: int64


We can see above that the class distribution is highly imbalanced, this would not lead to good sampling of the data for the classifier. For your learning, try using [SMOTE](https://imbalanced-learn.readthedocs.io/en/stable/api.html) to oversample the minority classes and then evaluate the performance with Naive Bayes and compare.